# BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.ipynb), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
%pip install faiss-cpu
%pip install google-search-results 
%pip install langchain 
%pip install openai
%pip install python-dotenv
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [3]:

from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [6]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain



from dotenv import load_dotenv
load_dotenv()
search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPAPI_API_KEY"))
search.run("hello world")

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)


tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective in python list format. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)




llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
# agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)



### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [8]:
OBJECTIVE = "Draft 5 and personalized emails to send to high schools in Massachusetts that have dog or dog breeds as their mascots asking them to contact their state representative to support a dog welfare bill. Format your emails to be personalized to each school. You can use the following template: Dear [school name], We are reaching out to [your school name] to ask you to contact your state representative to support a dog welfare bill. [Insert a paragraph about why you think this bill is important and why you think your school should support it. ]. I am writing to you because [insert a reason why you think your school should support this bill that mentions their mascot]. I hope you will consider contacting your state representative to support this bill [include contact information for the state representive for people who live near that school]. Thank you!"

In [9]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, task_execution_chain=agent_executor, verbose=verbose, max_iterations=max_iterations
)




In [10]:
task_list = todo_chain.run(OBJECTIVE)

task_list = task_list.split("\n\n")


In [11]:
%pip install rich
from rich import print


Note: you may need to restart the kernel to use updated packages.


In [ ]:
baby_agi({"objective": str(task_list)})